## Bitboard für Connect 4

[doc](https://towardsdatascience.com/creating-the-perfect-connect-four-ai-bot-c165115557b0)


Ein *board* modellieren wir als eine 6 x 7 numpy-Matrix. 


Die erste Reihe ist eine Sentinel-Reihe, die immer mit Nullen gefüllt ist.

Die Abbildung zeigt die Ordnung der Bits im Bitstring. 
 0 ist das least significant bit, d.h. das am weitesten rechts stehende Bit. Bits 6, 13, 20, 27, 34, 41 und 48 bilden eine *Sentinel*-Reihe (Wächter), die immer mit 0 gefüllt ist.

In [298]:
import numpy as np
board = np.zeros((7, 7), dtype = int)

for j in range(7):
    for i in range(6,-1,-1):
        board[i,j] = 6-i + 7 *j

for i in range(7):
    print(row(i,board))
    if i == 0:
        print('----------------------')

[ 6 13 20 27 34 41 48]
----------------------
[ 5 12 19 26 33 40 47]
[ 4 11 18 25 32 39 46]
[ 3 10 17 24 31 38 45]
[ 2  9 16 23 30 37 44]
[ 1  8 15 22 29 36 43]
[ 0  7 14 21 28 35 42]


#### Funktionen für das Board
Zum Füllen und Anzeigen des boards implementieren wir einige Funktionen



In [299]:
def makeBoard(i,j):
    return np.zeros((i, j), dtype = int)

def showBoard(board):
    for i in range(len(board)):
        print(*row(i,board))
        #print(''.join(str(board[x])))
    print('==============')
    print('0 1 2 3 4 5 6 ')
    print()
    
        
def row(i, board):
    return board[i,:]

def col(j, board):
    return board[:,j]

def insert(j, x, board):
    ''' insert in col j value x as in connect 4'''
    coltmp = np.argwhere(col(j,board)==0)  # 
    if len(coltmp) == 0: return False
    board[coltmp[-1],j] = x
    return True

def fill(j, s, board):
    ''' fill col j with values of s 
    eg. s = '1122' : 2 is at the bottom of col j '''
    for c in list(s)[::-1]:
        insert(j,int(c),board)

In [300]:
board = makeBoard(6,7)
fill(2,'2111', board)
fill(3,'12212', board)
fill(4,'22', board)
fill(5,'1', board)
showBoard(board)

0 0 0 0 0 0 0
0 0 0 1 0 0 0
0 0 2 2 0 0 0
0 0 1 2 0 0 0
0 0 1 1 2 0 0
0 0 1 2 2 1 0
0 1 2 3 4 5 6 



#### Board nach Bitstring

In [359]:
def board2bitnr(player, board):
    ''' returns int : zahl, die im bin-format das board repräsentiert'''
    position, mask = '', ''
    # Start with right-most column
    for j in range(6, -1, -1):
        # Add 0-bits to sentinel 
        mask += '0'
        position += '0'
        # Start with bottom row
        for i in range(0, 6):
            mask += ['0', '1'][int(board[i, j] != 0)]
            position += ['0', '1'][int(board[i, j] == player)]
    return int(position, 2), int(mask, 2)

def showBitnr(bn):
    bs = bitnr2str(bn)
    i, j = 6, 0
    board = makeBoard(7,7)
    for c in bs[::-1]:
        board[i,j] = int(c)
        i-=1
        if i == -1:
            j += 1
            i = 6 
    print()
    for i in range(7):
        print(*row(i,board))
        if i == 0:
            print('-'*13)
    print('=============')
    print('0 1 2 3 4 5 6')

def bitstr(bitnr):
    bitstr = bin(bitnr)[2:]
    return bitstr.rjust(49,'0')
  
    

#### Board mit Bitnummern, Bitstring und BitBoard

In [360]:
board = makeBoard(6,7)
fill(2,'2111', board)
fill(3,'12212', board)
fill(4,'22', board)
fill(5,'2', board)
showBoard(board)

bn1, mask = board2bitstring(1,board)
bn2 = bn1 ^ mask
bn1, bn2                         # bitnummer pro spieler
print("Bitnummern: ", bn1, bn2)
bs1, bs2  = bitstr(bn1), bitstr(bn2)       # bitstring pro spieler
print("Bitstrings: ", bs1, bs2)
showBitnr(bn1)
showBitnr(bn2)


0 0 0 0 0 0 0
0 0 0 1 0 0 0
0 0 2 2 0 0 0
0 0 1 2 0 0 0
0 0 1 1 2 0 0
0 0 1 2 2 2 0
0 1 2 3 4 5 6 

Bitnummern:  37863424 35192438784
Bitstrings:  0000000000000000000000010010000011100000000000000 0000000000000100000110001101000100000000000000000

0 0 0 0 0 0 0
-------------
0 0 0 0 0 0 0
0 0 0 1 0 0 0
0 0 0 0 0 0 0
0 0 1 0 0 0 0
0 0 1 1 0 0 0
0 0 1 0 0 0 0
0 1 2 3 4 5 6

0 0 0 0 0 0 0
-------------
0 0 0 0 0 0 0
0 0 0 0 0 0 0
0 0 1 1 0 0 0
0 0 0 1 0 0 0
0 0 0 0 1 0 0
0 0 0 1 1 1 0
0 1 2 3 4 5 6


#### Check auf Vierer

Die Funktion entdeckt, ob vier Positionen verbunden sind

In [361]:
def check4(bn):
    '''  bn: bitnummer '''
    # Horizontal  
    m = bn & (bn >> 7)
    if m & (m >> 14):
        return True
    # Diagonal \
    m = bn & (bn >> 6)
    if m & (m >> 12):
        return True
    # Diagonal /
    m = bn & (bn >> 8)
    if m & (m >> 16):
        return True
    # Vertical
    m = bn & (bn >> 1)
    if m & (m >> 2):
        return True
    # Nothing found
    return False

In [362]:
check4(bn1), check4(bn2)

(False, True)

#### Erläuterung des horizontalen Checks

Wir betrachten folgendes bitboard

In [388]:
board = makeBoard(6,7)
board[2] = np.array([0,0,1,1,1,1,0])

bn = board2bitnr(1,board)[0]
showBitnr(bn)



0 0 0 0 0 0 0
-------------
0 0 0 0 0 0 0
0 0 0 0 0 0 0
0 0 1 1 1 1 0
0 0 0 0 0 0 0
0 0 0 0 0 0 0
0 0 0 0 0 0 0
0 1 2 3 4 5 6


Die Bits in *bn1* sind um 7 Stellen nach rechts verschoben. 

In [389]:
bn1 = bn >> 7
print(bitstr(bn))
print(bitstr(bn1))

0000000000100000010000001000000100000000000000000
0000000000000000010000001000000100000010000000000


Das bedeutet, dass die Bits im Board um eine Stelle nach links verschoben wurden.

In [390]:
showBitnr(bn)
showBitnr(bn1)


0 0 0 0 0 0 0
-------------
0 0 0 0 0 0 0
0 0 0 0 0 0 0
0 0 1 1 1 1 0
0 0 0 0 0 0 0
0 0 0 0 0 0 0
0 0 0 0 0 0 0
0 1 2 3 4 5 6

0 0 0 0 0 0 0
-------------
0 0 0 0 0 0 0
0 0 0 0 0 0 0
0 1 1 1 1 0 0
0 0 0 0 0 0 0
0 0 0 0 0 0 0
0 0 0 0 0 0 0
0 1 2 3 4 5 6


Mit der *&*-Verbindung prüfen wir also, ob an einer Stelle dasselbe steht, wie an der Stelle rechts daneben

In [391]:
m = bn & bn1
showBitnr(m)


0 0 0 0 0 0 0
-------------
0 0 0 0 0 0 0
0 0 0 0 0 0 0
0 0 1 1 1 0 0
0 0 0 0 0 0 0
0 0 0 0 0 0 0
0 0 0 0 0 0 0
0 1 2 3 4 5 6


Dieses Bitboard können wir wie folgt interpretieren: Dort wo eine 1 steht, steht im ursprünglichen bitboard links daneben auch eine 1

In [392]:
m1 = m >> 14
print(bitstr(m))
print(bitstr(m1))

0000000000000000010000001000000100000000000000000
0000000000000000000000000000000100000010000001000


In [393]:
showBitnr(m)
showBitnr(m1)


0 0 0 0 0 0 0
-------------
0 0 0 0 0 0 0
0 0 0 0 0 0 0
0 0 1 1 1 0 0
0 0 0 0 0 0 0
0 0 0 0 0 0 0
0 0 0 0 0 0 0
0 1 2 3 4 5 6

0 0 0 0 0 0 0
-------------
0 0 0 0 0 0 0
0 0 0 0 0 0 0
1 1 1 0 0 0 0
0 0 0 0 0 0 0
0 0 0 0 0 0 0
0 0 0 0 0 0 0
0 1 2 3 4 5 6


In [394]:
showBitnr(m & m1)


0 0 0 0 0 0 0
-------------
0 0 0 0 0 0 0
0 0 0 0 0 0 0
0 0 1 0 0 0 0
0 0 0 0 0 0 0
0 0 0 0 0 0 0
0 0 0 0 0 0 0
0 1 2 3 4 5 6
